# Keras 101: A simple Neural Network for House Pricing regression


In this post, we will be covering some basics of data exploration and building a model with Keras in order to help us on predicting the selling price of a given house in the Boston (MA) area. As an application of this model in the real world, you can think about being a real state agent looking for a tool to help you on your day-to-day duties, which for me, at least, sounds pretty good when compared to just gut-estimation.

For this exercise, we will be using the [Plotly](https://plot.ly/python/) library instead of the good ol' fashioned matplotlib, due to having more interactive plots, which for sure help in understanding the data. We will also use the [Scikit-Learn](https://scikit-learn.org/stable/) and [Keras](https://keras.io/) for building the models, [Pandas](https://pandas.pydata.org/) library to manipulate our data and the [SHAP library](https://github.com/slundberg/shap) to generate explanations for our trained model.

### Importing the dataset

In this example, we wil be using the sklearn.datasets module, which contains the Boston dataset. You could also use the keras.datasets module, but this one does not contain the labels of the features, so we decided to use scikits one. Let's also convert it to a Pandas DataFrame and print it's head.




In [ ]:
from sklearn.datasets import load_boston
import pandas as pd

boston_dataset = load_boston()

df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df['MEDV'] = boston_dataset.target
df = df.iloc[250:,:]
df.head(n=10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
250,0.14030,22.0,5.86,0.0,0.431,6.487,13.0,7.3967,7.0,330.0,19.1,396.28,5.90,24.4
251,0.21409,22.0,5.86,0.0,0.431,6.438,8.9,7.3967,7.0,330.0,19.1,377.07,3.59,24.8
252,0.08221,22.0,5.86,0.0,0.431,6.957,6.8,8.9067,7.0,330.0,19.1,386.09,3.53,29.6
253,0.36894,22.0,5.86,0.0,0.431,8.259,8.4,8.9067,7.0,330.0,19.1,396.90,3.54,42.8
254,0.04819,80.0,3.64,0.0,0.392,6.108,32.0,9.2203,1.0,315.0,16.4,392.89,6.57,21.9
255,0.03548,80.0,3.64,0.0,0.392,5.876,19.1,9.2203,1.0,315.0,16.4,395.18,9.25,20.9
256,0.01538,90.0,3.75,0.0,0.394,7.454,34.2,6.3361,3.0,244.0,15.9,386.34,3.11,44.0
257,0.61154,20.0,3.97,0.0,0.647,8.704,86.9,1.8010,5.0,264.0,13.0,389.70,5.12,50.0
258,0.66351,20.0,3.97,0.0,0.647,7.333,100.0,1.8946,5.0,264.0,13.0,383.29,7.79,36.0
259,0.65665,20.0,3.97,0.0,0.647,6.842,100.0,2.0107,5.0,264.0,13.0,391.93,6.90,30.1



### Exploratory Data Analysis

Making yourself comfortable and familiar with your dataset is a fundamental step to help you comprehend your data and draw better conclusions and explanations from your results.

Initially, let's plot a few box plots, which will help us to better visualizate anomalies and/or outliers in data distribution. If you are confused about what is a box plot and how it can help us to better visualizate the distribution of our data, here is a brief description from Ross (1977):

> In descriptive statistics, a box plot is a method for graphically depicting groups of numerical data through their quartiles. Box plots may also have lines extending vertically from the boxes (whiskers) indicating variability outside the upper and lower quartiles, hence the terms box-and-whisker plot and box-and-whisker diagram. Outliers may be plotted as individual points.


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math

total_items = len(df.columns)
items_per_row = 3
total_rows = math.ceil(total_items / items_per_row)

fig = make_subplots(rows=total_rows, cols=items_per_row)

cur_row = 1
cur_col = 1

for index, column in enumerate(df.columns):
    fig.add_trace(go.Box(y=df[column], name=column), row=cur_row, col=cur_col)
    
    if cur_col % items_per_row == 0:
        cur_col = 1
        cur_row = cur_row + 1
    else:
        cur_col = cur_col + 1
    

fig.update_layout(height=1000, width=550,  showlegend=False)
fig.show()

These results do corroborate our initial assumptions about having outliers in some columns. Let's also plot some scatter plots for each feature and the target variable, as well as their intercept lines:


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
import numpy as np

total_items = len(df.columns)
items_per_row = 3
total_rows = math.ceil(total_items / items_per_row)

fig = make_subplots(rows=total_rows, cols=items_per_row, subplot_titles=df.columns)

cur_row = 1
cur_col = 1

for index, column in enumerate(df.columns):
    fig.add_trace(go.Scattergl(x=df[column], 
                            y=df['MEDV'], 
                            mode="markers", 
                            marker=dict(size=3)), 
                  row=cur_row, 
                  col=cur_col)
    
    intercept = np.poly1d(np.polyfit(df[column], df['MEDV'], 1))(np.unique(df[column]))
    
    fig.add_trace(go.Scatter(x=np.unique(df[column]), 
                             y=intercept, 
                             line=dict(color='red', width=1)), 
                  row=cur_row, 
                  col=cur_col)
    
    if cur_col % items_per_row == 0:
        cur_col = 1
        cur_row = cur_row + 1
    else:
        cur_col = cur_col + 1
    

fig.update_layout(height=1000, width=550, showlegend=False)
fig.show()


From this initial data exploration, we can have two major conclusions:

- There is a strong linear correlation between the RM (average number of rooms per dwelling) and LSTAT (% lower status of the population) with the target variable, being the RM a positive and the LSTAT a negative correlation.
- There are some records containing outliers, which we could preprocess in order to input our model with more normalized data.

### Data preprocessing

Before we proceed into any data preprocessing, it's important to split our data into training and test sets. We should not apply any kind of preprocessing into our data without taking into account that we should not leak information from our test set. For this step, we can use the *train_test_split* method from scikit-learn. In this case, we will use a split of 70% of the data for training and 30% for testing. We also set a random_state seed, in order to allow reprocibility.


In [ ]:
from sklearn.model_selection import train_test_split

X = df.loc[:, df.columns != 'MEDV']
y = df.loc[:, df.columns == 'MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In order to provide a standardized input to our neural network, we need the perform the normalization of our dataset. This can be seen as an step to reduce the differences in scale that may arise from the existent features. We perform this normalization by subtracting the mean from our data and dividing it by the standard deviation. **One more time,  this normalization should only be performed by using the mean and standard deviation from the training set, in order to avoid any information leak from the test set.**


In [ ]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std


### Build our model

Due to the small amount of presented data in this dataset, we must be careful to not create an overly complex model, which could lead to overfitting our data. For this, we are going to adopt an architecture based on two Dense layers, the first with 128 and the second with 64 neurons, both using a ReLU activation function. A dense layer with a linear activation will be used as output layer.

In order to allow us to know if our model is properly learning, we will use a mean squared error loss function and to report the performance of it we will adopt the mean average error metric.

By using the summary method from Keras, we can see that we have a total of 5,121 parameters, which is acceptable for us.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(128, input_shape=(13, ), activation='relu', name='dense_1'))
model.add(Dense(64, activation='relu', name='dense_2'))
model.add(Dense(1, activation='linear', name='dense_output'))

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               1792      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_output (Dense)        (None, 1)                 65        
                                                                 
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________



### Train our model

This step is pretty straightforward: fit our model with both our features and their labels, for a total amount of 100 epochs, separating 5% of the samples (18 records) as validation set.


In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_split=0.05)

Epoch 1/100
6/6 [==============================] - 1s 45ms/step - loss: 505.8154 - mae: 20.3139 - val_loss: 667.4455 - val_mae: 23.8711
Epoch 2/100
6/6 [==============================] - 0s 7ms/step - loss: 476.6905 - mae: 19.6023 - val_loss: 639.3269 - val_mae: 23.3226
Epoch 3/100
6/6 [==============================] - 0s 7ms/step - loss: 451.9773 - mae: 18.9726 - val_loss: 612.1458 - val_mae: 22.7868
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 426.5397 - mae: 18.3077 - val_loss: 581.3043 - val_mae: 22.1709
Epoch 5/100
6/6 [==============================] - 0s 7ms/step - loss: 397.9704 - mae: 17.5032 - val_loss: 542.7927 - val_mae: 21.3721
Epoch 6/100
6/6 [==============================] - 0s 7ms/step - loss: 362.4959 - mae: 16.5184 - val_loss: 495.3499 - val_mae: 20.3382
Epoch 7/100
6/6 [==============================] - 0s 7ms/step - loss: 319.6630 - mae: 15.2927 - val_loss: 438.7738 - val_mae: 19.0231
Epoch 8/100
6/6 [==============================] - 0s 

## Saving Architechture and Weights:

In [ ]:
from keras.models import model_from_json
# serialize model to json
json_model = model.to_json()
#json_model
#save the model architecture to JSON file
with open('client2_nn.json', 'w') as json_file:
    json_file.write(json_model)

In [ ]:
model.save_weights('client2_nn.h5')

In [ ]:
weight_layer_1=model.layers[0].get_weights()[0]
bias_layer_1=model.layers[0].get_weights()[1]

weight_layer_2=model.layers[1].get_weights()[0]
bias_layer_2=model.layers[1].get_weights()[1]

weight_layer_3=model.layers[2].get_weights()[0]
bias_layer_3=model.layers[2].get_weights()[1]




In [ ]:

print("Weights and biases of the layers before training the model: \n")
count=0
ls=[]
for layer in model.layers:
  print(layer.name)
  print("Weights")
  print("Shape: ",layer.get_weights()[0].shape,'\n',layer.get_weights()[0])
  print("Bias")
  print("Shape: ",layer.get_weights()[1].shape,'\n',layer.get_weights()[1],'\n')
  count+=1


Weights and biases of the layers before training the model: 

dense_1
Weights
Shape:  (13, 128) 
 [[-0.20620233  0.01386485 -0.12644447 ... -0.01013972 -0.12886311
  -0.2205407 ]
 [-0.04403112 -0.21524084 -0.18015893 ...  0.05163318 -0.3050355
   0.03850208]
 [-0.00263445 -0.0642998   0.17050117 ...  0.00907096 -0.19724306
   0.20542373]
 ...
 [-0.03869863  0.10160216 -0.15584362 ... -0.16964395 -0.11670371
   0.20025706]
 [ 0.09101109  0.09820924  0.1439471  ...  0.13530692 -0.00099798
   0.0198123 ]
 [-0.19771607 -0.2711317  -0.173123   ...  0.13911784  0.02175527
  -0.03676476]]
Bias
Shape:  (128,) 
 [ 0.05279677  0.04253523  0.05065763  0.09357475  0.0856774   0.07139343
  0.12061371  0.1277975   0.10063565  0.12493535  0.09920354  0.11022834
  0.09857628  0.14967862  0.11294523 -0.05662919  0.11521484  0.07108802
  0.07898472  0.07973462  0.05294615  0.01201571  0.07127032  0.09101413
  0.04138983  0.03691722  0.08592107  0.10957033  0.07091609  0.05709133
  0.04714563  0.07758905

In [ ]:
w,bias=model.layers[0].get_weights()
w=np.asarray(w)
w

array([[-0.20620233,  0.01386485, -0.12644447, ..., -0.01013972,
        -0.12886311, -0.2205407 ],
       [-0.04403112, -0.21524084, -0.18015893, ...,  0.05163318,
        -0.3050355 ,  0.03850208],
       [-0.00263445, -0.0642998 ,  0.17050117, ...,  0.00907096,
        -0.19724306,  0.20542373],
       ...,
       [-0.03869863,  0.10160216, -0.15584362, ..., -0.16964395,
        -0.11670371,  0.20025706],
       [ 0.09101109,  0.09820924,  0.1439471 , ...,  0.13530692,
        -0.00099798,  0.0198123 ],
       [-0.19771607, -0.2711317 , -0.173123  , ...,  0.13911784,
         0.02175527, -0.03676476]], dtype=float32)

By plotting both loss and mean average error, we can see that our model was capable of learning patterns in our data without overfitting taking place (as shown by the validation set curves):


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_loss'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()


### Evaluate our model


In [ ]:
mse_nn, mae_nn = model.evaluate(X_test, y_test)

print('Mean squared error on test data: ', mse_nn)
print('Mean absolute error on test data: ', mae_nn)

3/3 [==============================] - 0s 8ms/step - loss: 22.7002 - mae: 3.0808
Mean squared error on test data:  22.70024871826172
Mean absolute error on test data:  3.0807809829711914


# Retraining with updated model co-efficients:

## Loading the Previous Architecture of Model:


# Importing Model 1 weights:

In [ ]:
import os
import tensorflow as tf
import keras

In [ ]:
from keras.initializers import glorot_uniform

# Creating a new model from the saved JSON file
# reda the model from the JSOn file
with open('client2_nn.json', 'r') as json_file:
    json_savedModel= json_file.read()
#json_savedModel
#load the model architecture 
model_2 = tf.keras.models.model_from_json(json_savedModel)
model_2.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               1792      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_output (Dense)        (None, 1)                 65        
                                                                 
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________


## Importing the List of numpy arrays of Updated Weights

In [ ]:
import pickle
import numpy as np

with open("updated_layer_1.json", "rb") as fp1:
  ls1=pickle.load(fp1)
with open("updated_layer_2.json", "rb") as fp2:
  ls2=pickle.load(fp2)
with open("updated_layer_3.json", "rb") as fp3:
  ls3=pickle.load(fp3)

In [ ]:
print("Weight , Bias of layer 1")
np.shape(ls1[0]),np.shape(ls1[1])

Weight , Bias of layer 1


((13, 128), (128,))

In [ ]:
print("Weight , Bias of layer 2")
np.shape(ls2[0]),np.shape(ls2[1])

Weight , Bias of layer 2


((128, 64), (64,))

In [ ]:
print("Weight , Bias of layer 3")
np.shape(ls3[0]),np.shape(ls3[1])

Weight , Bias of layer 3


((64, 1), (1,))

## Setting the updated weights in model

In [ ]:
model_2.layers[0].set_weights(ls1)

In [ ]:
model_2.layers[1].set_weights(ls2)

In [ ]:
model_2.layers[2].set_weights(ls3)

In [ ]:
print("Round 1 of Averaging: Weights and biases of the layers before training : \n")
count=0
ls=[]
for layer in model_2.layers:
  print(layer.name)
  print("Weights")
  print("Shape: ",layer.get_weights()[0].shape,'\n',layer.get_weights()[0])
  print("Bias")
  print("Shape: ",layer.get_weights()[1].shape,'\n',layer.get_weights()[1],'\n')
  count+=1

Round 1 of Averaging: Weights and biases of the layers before training : 

dense_1
Weights
Shape:  (13, 128) 
 [[ 6.20943308e-02 -1.78483278e-01 -1.02627665e-01 ...  2.57962830e-02
  -7.57881925e-02 -1.13780670e-01]
 [-3.46610397e-02  6.03686534e-02 -1.22795708e-01 ...  1.10384226e-02
   1.25830933e-01 -3.39010730e-04]
 [ 9.89168361e-02 -1.88106611e-01  1.48603730e-02 ...  1.31532550e-04
  -2.36775130e-02 -1.26437113e-01]
 ...
 [ 3.86220217e-03  2.81191356e-02 -2.66750515e-01 ... -1.85343191e-01
   7.72679448e-02  9.53996703e-02]
 [-1.08798683e-01  1.21338507e-02  2.08816499e-01 ...  1.49667889e-01
   2.09382176e-02 -5.49083985e-02]
 [-5.45909479e-02  3.19619589e-02 -1.03366733e-01 ... -3.93435098e-02
   4.24711704e-02  1.92250177e-01]]
Bias
Shape:  (128,) 
 [0.05488506 0.08146667 0.12298182 0.12015612 0.07622987 0.10471848
 0.09643918 0.12511289 0.06834874 0.12589926 0.08692739 0.10243049
 0.14626242 0.07084406 0.07910706 0.09141494 0.02152244 0.08845909
 0.10820374 0.09974018 0.09458

In [ ]:
client1_weight_layer_1=model_1.layers[0].get_weights()[0]
client1_bias_layer_1=model_1.layers[0].get_weights()[1]

client1_weight_layer_2=model_1.layers[1].get_weights()[0]
client1_bias_layer_2=model_1.layers[1].get_weights()[1]

client1_weight_layer_3=model_1.layers[2].get_weights()[0]
client1_bias_layer_3=model_1.layers[2].get_weights()[1]


# Now Retraing with the updated models on same data:


In [ ]:
model_2.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               1792      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_output (Dense)        (None, 1)                 65        
                                                                 
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model_2.fit(X_train, y_train, epochs=100, validation_split=0.05)

Epoch 1/100
6/6 [==============================] - 1s 41ms/step - loss: 90.2639 - mae: 7.8071 - val_loss: 75.6807 - val_mae: 7.4397
Epoch 2/100
6/6 [==============================] - 0s 7ms/step - loss: 52.2233 - mae: 5.3443 - val_loss: 37.7333 - val_mae: 4.9063
Epoch 3/100
6/6 [==============================] - 0s 8ms/step - loss: 30.9795 - mae: 3.5639 - val_loss: 20.9792 - val_mae: 4.1374
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 24.5762 - mae: 3.3170 - val_loss: 14.7427 - val_mae: 3.2749
Epoch 5/100
6/6 [==============================] - 0s 7ms/step - loss: 22.5368 - mae: 3.3802 - val_loss: 10.6716 - val_mae: 2.7082
Epoch 6/100
6/6 [==============================] - 0s 7ms/step - loss: 20.2254 - mae: 3.1984 - val_loss: 8.9305 - val_mae: 2.4588
Epoch 7/100
6/6 [==============================] - 0s 7ms/step - loss: 19.1441 - mae: 3.0440 - val_loss: 8.5621 - val_mae: 2.2402
Epoch 8/100
6/6 [==============================] - 0s 7ms/step - loss: 18.2714 - mae

### Evaluate our model


In [ ]:
mse_nn, mae_nn = model_2.evaluate(X_test, y_test)

print('Mean squared error on test data: ', mse_nn)
print('Mean absolute error on test data: ', mae_nn)

3/3 [==============================] - 0s 6ms/step - loss: 27.3704 - mae: 3.0374
Mean squared error on test data:  27.370370864868164
Mean absolute error on test data:  3.0374350547790527
